[View in Colaboratory](https://colab.research.google.com/github/Joovvhan/ColabTest/blob/master/Transfer_Learning_Model_Comparison_DenseNet169.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wf

In [3]:
os.listdir('gdrive/My Drive/Colab')

['Model', 'Data']

In [0]:
dataPath = 'gdrive/My Drive/Colab/Data'

"""
folders = os.listdir(dataPath)

for folder in folders:
  files = os.listdir(dataPath + '/' + folder)
  print(len(files))
  print(files[-1])
"""

folderF1 = dataPath + '/' + 'A3F1P3'  
folderF5 = dataPath + '/' + 'A3F5P3'
filesF1 = os.listdir(folderF1)
filesF5 = os.listdir(folderF5)

folderA5 = dataPath + '/' + 'A5F3P3'
filesA5 = os.listdir(folderA5)

In [5]:
filesF1[0]

'A3.490879_F0.139398_P2.076554_H360.426598.wav'

In [0]:
fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[0])
dataFloat = dataInt16 / (2 ** 15)

fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[0])
dataFloat = dataInt16 / (2 ** 15)

In [0]:
nsc = 1470
nov = nsc/2
nff = nsc 

In [0]:
Pxx, freqs, bins, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                   window=np.hamming(nsc), cmap='viridis')
plt.close()
img = Pxx[0:224, :]

plt.imshow(10*np.log10(img), cmap='viridis')
plt.close()

In [0]:
import time

In [0]:
from tqdm import tqdm, trange

In [11]:
imgSize = 224
imgsF1 = np.zeros([len(filesF1), imgSize, imgSize])

for i in trange(len(filesF1)):
    fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF1[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|██████████| 1000/1000 [01:19<00:00, 12.54it/s]


In [12]:
imgsF5 = np.zeros([len(filesF5), imgSize, imgSize])

for i in trange(len(filesF5)):
    fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF5[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|██████████| 1000/1000 [01:20<00:00, 11.23it/s]


In [0]:
trainIdxF1 = np.random.choice(len(imgsF1) - 1, int(len(imgsF1) * 0.8), replace=False)
testIdxF1 = list(set(range(0, len(imgsF1))) - set(trainIdxF1))

trainImgsF1 = imgsF1[trainIdxF1, :, :]
testImgsF1 = imgsF1[testIdxF1, :, :]

trainIdxF5 = np.random.choice(len(imgsF5) - 1, int(len(imgsF5) * 0.8), replace=False)
testIdxF5 = list(set(range(0, len(imgsF5))) - set(trainIdxF5))

trainImgsF5 = imgsF5[trainIdxF5, :, :]
testImgsF5 = imgsF5[testIdxF5, :, :]

In [0]:
trainImgs = np.vstack([trainImgsF1, trainImgsF5])
testImgs = np.vstack([testImgsF1, testImgsF5])

In [0]:
trainMean = np.mean(trainImgs)
trainStd = np.std(trainImgs)

In [0]:
trainImgsNorm = (trainImgs - trainMean) / trainStd
testImgsNorm = (testImgs - trainMean) / trainStd

trainImgsNorm = trainImgsNorm.reshape(list(trainImgsNorm.shape) + [1])
testImgsNorm = testImgsNorm.reshape(list(testImgsNorm.shape) + [1])

In [0]:
X_train = np.stack([trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0]], axis = -1)
X_test = np.stack([testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0]], axis = -1)

In [0]:
trainLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.8)), np.zeros( int(len(imgsF1) * 0.8))), axis = -1)
testLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.2)), np.zeros( int(len(imgsF1) * 0.2))), axis = -1)

trainLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.8)), np.ones( int(len(imgsF5) * 0.8))), axis = -1)
testLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.2)), np.ones( int(len(imgsF5) * 0.2))), axis = -1)

Y_train = np.vstack((trainLabelF1, trainLabelF5))
Y_test = np.vstack((testLabelF1, testLabelF5))

In [19]:
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential

Using TensorFlow backend.


In [0]:
from keras.applications.densenet import DenseNet169
from keras import backend as K

for i in range(1000):

    input_tensor = Input(shape=(224, 224, 3))

    modelDenseNet169WoTop = None
    modelDenseNet169 = None

    modelDenseNet169WoTop = DenseNet169(input_tensor=input_tensor, weights='imagenet', include_top=False)

    # modelDenseNet169WoTop.summary()

    modelDenseNet169 = Sequential()

    modelDenseNet169.add(modelDenseNet169WoTop)

    modelDenseNet169.add(GlobalAveragePooling2D())

    modelDenseNet169.add(Dense(2, activation='softmax'))

#     modelDenseNet169.summary()

    modelDenseNet169.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    modelDenseNet169.fit(X_train, Y_train,
              batch_size=4, epochs=20, verbose=1,
              validation_data=(X_test, Y_test))

    K.clear_session()
    
    # modelDenseNet201.summary()



# modelXceptionWoTop.summary()

# modelXception = Xception(input_tensor=input_tensor, weights='imagenet')

# modelXception.summary()

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 207s 130ms/step - loss: 0.1884 - acc: 0.9494 - val_loss: 9.7587e-05 - val_acc: 1.0000
Epoch 2/20
1600/1600 [==============================] - 167s 104ms/step - loss: 0.1376 - acc: 0.9244 - val_loss: 1.2495e-05 - val_acc: 1.0000
Epoch 3/20
1600/1600 [==============================] - 167s 104ms/step - loss: 0.1120 - acc: 0.9425 - val_loss: 3.5566e-06 - val_acc: 1.0000
Epoch 4/20
1600/1600 [==============================] - 166s 104ms/step - loss: 0.1024 - acc: 0.9456 - val_loss: 6.6735e-06 - val_acc: 1.0000
Epoch 5/20
1600/1600 [==============================] - 168s 105ms/step - loss: 0.1052 - acc: 0.9475 - val_loss: 3.8696e-06 - val_acc: 1.0000
Epoch 6/20
1600/1600 [==============================] - 167s 104ms/step - loss: 0.0883 - acc: 0.9544 - val_loss: 1.4035e-06 - val_acc: 1.0000
Epoch 7/20
1600/1600 [==============================] - 166s 104ms/step - loss: 0.0808 - acc: 0.9575 

In [0]:
from keras.applications.densenet import DenseNet201
from keras import backend as K

for i in range(1000):

    input_tensor = Input(shape=(224, 224, 3))

    modelDenseNet201WoTop = None
    modelDenseNet201 = None

    modelDenseNet201WoTop = DenseNet201(input_tensor=input_tensor, weights='imagenet', include_top=False)

    # modelDenseNet201WoTop.summary()

    modelDenseNet201 = Sequential()

    modelDenseNet201.add(modelDenseNet201WoTop)

    modelDenseNet201.add(GlobalAveragePooling2D())

    modelDenseNet201.add(Dense(2, activation='softmax'))

    modelDenseNet201.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    modelDenseNet201.fit(X_train, Y_train,
              batch_size=4, epochs=20, verbose=1,
              validation_data=(X_test, Y_test))

#   modelDenseNet201.summary()

    K.clear_session()

76398592/76391848 [==============================] - 12s 0us/step
Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 256s 160ms/step - loss: 0.2072 - acc: 0.9131 - val_loss: 3.5324e-05 - val_acc: 1.0000
Epoch 2/20
1600/1600 [==============================] - 194s 121ms/step - loss: 0.1416 - acc: 0.9388 - val_loss: 8.5024e-05 - val_acc: 1.0000
Epoch 3/20
1600/1600 [==============================] - 193s 121ms/step - loss: 0.1316 - acc: 0.9244 - val_loss: 1.2772e-04 - val_acc: 1.0000
Epoch 4/20
1600/1600 [==============================] - 194s 121ms/step - loss: 0.1216 - acc: 0.9388 - val_loss: 1.3892e-04 - val_acc: 1.0000
Epoch 5/20
1600/1600 [==============================] - 194s 121ms/step - loss: 0.1186 - acc: 0.9237 - val_loss: 4.7254e-05 - val_acc: 1.0000
Epoch 6/20
1600/1600 [==============================] - 194s 121ms/step - loss: 0.1176 - acc: 0.9163 - val_loss: 3.6712e-05 - val_acc: 1.0000
Epoch 7/20
1600/1600 [=============

In [5]:
modelDenseNet169 = DenseNet169(input_tensor=input_tensor, weights='imagenet')

modelDenseNet169.summary()

59850752/59842792 [==============================] - 6s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K

for i in range(100):

    input_tensor = Input(shape=(224, 224, 3))
    
    modelInceptionV3WoTop = None
    modelInceptionV3 = None

    modelInceptionV3WoTop = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
    # modelInceptionV3WoTop.summary()
    # modelInceptionV3 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
    # modelInceptionV3.summary()

    modelInceptionV3 = Sequential()

    modelInceptionV3.add(modelInceptionV3WoTop)
    modelInceptionV3.add(GlobalAveragePooling2D())
    modelInceptionV3.add(Dense(2, activation='softmax'))

    # modelInceptionV3.summary()

    modelInceptionV3.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    modelInceptionV3.fit(X_train, Y_train,
              batch_size=4, epochs=6, verbose=1,
              validation_data=(X_test, Y_test))
    
    K.clear_session()

In [0]:
from keras.applications import VGG19

from keras import backend as K

for i in range(100):
    
    input_tensor = Input(shape=(224, 224, 3))
    
    modelVGG19WoTop = None
    modelVGG19 = None

    modelVGG19WoTop = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

    modelVGG19 = Sequential()

    modelVGG19.add(modelVGG19WoTop)
    modelVGG19.add(Flatten())
    modelVGG19.add(Dense(4096, activation='relu'))
    modelVGG19.add(Dropout(0.5))
    modelVGG19.add(Dense(4096, activation='relu'))
    modelVGG19.add(Dropout(0.5))
    
#     modelVGG19.add(Dense(128, activation='relu'))
#     modelVGG19.add(Dropout(0.5))
#     modelVGG19.add(Dense(128, activation='relu'))
#     modelVGG19.add(Dropout(0.5))
    
    modelVGG19.add(Dense(2, activation='softmax'))

    modelVGG19.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    modelVGG19.fit(X_train, Y_train,
              batch_size=2, epochs=8, verbose=1,
              validation_data=(X_test, Y_test))
    
    K.clear_session()

In [0]:
from keras.applications import VGG16

input_tensor = Input(shape=(224, 224, 3))

modelVGG16WoTop = None
modelVGG16 = None

modelVGG16WoTop = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelVGG16 = Sequential()

modelVGG16.add(modelVGG16WoTop)
modelVGG16.add(Flatten())
modelVGG16.add(Dense(100, activation='relu'))
modelVGG16.add(Dropout(0.5))
modelVGG16.add(Dense(100, activation='relu'))
modelVGG16.add(Dropout(0.5))

modelVGG16.add(Dense(2, activation='softmax'))

modelVGG16.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelVGG16.fit(X_train, Y_train,
          batch_size=2, epochs=2, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

modelResNet50WoTop = None
modelResNet50 = None

input_tensor = Input(shape=(224, 224, 3))
modelResNet50WoTop = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelResNet50 = Sequential()

modelResNet50.add(modelResNet50WoTop)
modelResNet50.add(Flatten())
modelResNet50.add(Dense(2, activation='softmax'))

modelResNet50.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelResNet50.fit(X_train, Y_train,
          batch_size=4, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import datetime

now = datetime.datetime.now()
modelResNet50.save('gdrive/My Drive/Colab/Model/ResNet50 {}.h5'.format(now.strftime('%m-%d %H:%M:%S')))
# modelResNet50_ = load_model('gdrive/My Drive/Colab/Model/ResNet50.h5')

# Y_pred = modelResNet50_.predict(X_test)
# print(confusion_matrix(np.argmax(Y_pred, axis = 1), np.argmax(Y_test, axis = 1)))

In [0]:
# for i in range(1000, 1010):
#     plt.figure()
#     plt.imshow(X_train[i, :, :, 0]);
#     plt.plot()

